In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

In [2]:
import numpy as np
import pickle as pic

from sklearn.model_selection import train_test_split

In [3]:
import deep_learning

In [4]:
if True:
    with open("splitted_rws.pic", "rb") as r:
        X_training, X_val, X_extraction = pic.load(r)
        X_training, X_val, X_extraction = X_training[:, 14790:20060], X_val[:, 14790:20060], X_extraction[:, 14790:20060]
    with open("splitted_labels_1000000.pic", "rb") as r:
        y_training, y_val, y_extraction = pic.load(r)
else:
    with open("traces_rws_only.pic", "rb") as r:
        traces_rws_only = pic.load(r)
    with open("labels_1000000.pic", "rb") as r:
        rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels = pic.load(r)

    X_total, y_total = deep_learning.prepare_data_dl(traces_rws_only, round_perms_labels, copy_perms_labels, round_masks_labels, rws_perms_labels, rws_masks_labels)

    profile, test = train_test_split(np.arange(X_total.shape[0]), train_size=750_000, random_state=0)

    X_profiling, X_extraction = X_total[profile], X_total[test]
    train, val = train_test_split(np.arange(X_profiling.shape[0]), test_size=0.1, random_state=0)
    X_training, X_val = X_profiling[train], X_profiling[val]

    with open("splitted_rws.pic", "wb") as w:
        pic.dump((X_training, X_val, X_extraction), w)

    if False:
        y_profiling = {}
        y_training = {}
        y_val = {}
        y_extraction = {}
        for label in y_total.keys():
            print(label, end="\r")
            y_profiling[label], y_extraction[label] = y_total[label][profile], y_total[label][test]
            y_training[label], y_val[label] = y_profiling[label][train], y_profiling[label][val]

        with open("splitted_labels_1000000.pic", "wb") as w:
            pic.dump((y_training, y_val, y_extraction), w)
    else:
        with open("splitted_labels_1000000.pic", "rb") as r:
            y_training, y_val, y_extraction = pic.load(r)


In [5]:
resnet = deep_learning.ResNetSCA(network="orig_rws_3", epochs=1000, dataset_size=X_training.shape[0])
try:
    deep_learning.check_file_exists("./resnet_models/resnet_750000_orig_rws_3.keras")
    from tensorflow.keras.models import load_model
    resnet.model = load_model("./resnet_models/resnet_750000_orig_rws_3.keras")
except ValueError:
    pass

In [6]:
train_gen = deep_learning.DataGenerator(X_training, y_training)
val_gen = deep_learning.DataGenerator(X_val, y_val)

In [7]:
history = resnet.train_model_generator(train_gen, val_gen, "./resnet_models/resnet_750000_orig_rws_3.keras", patience=10)

Epoch 1/1000
10547/10547 [==============================] - 4601s 433ms/step - loss: 146.0387 - rws_mask_72_0_output_loss: 2.8079 - rws_mask_72_1_output_loss: 2.8081 - rws_mask_73_0_output_loss: 2.8105 - rws_mask_73_1_output_loss: 2.8100 - rws_mask_74_0_output_loss: 2.8098 - rws_mask_74_1_output_loss: 2.8075 - rws_mask_75_0_output_loss: 2.8087 - rws_mask_75_1_output_loss: 2.8088 - rws_mask_76_0_output_loss: 2.8100 - rws_mask_76_1_output_loss: 2.8082 - rws_mask_77_0_output_loss: 2.8074 - rws_mask_77_1_output_loss: 2.8094 - rws_mask_78_0_output_loss: 2.8081 - rws_mask_78_1_output_loss: 2.8075 - rws_mask_79_0_output_loss: 2.8048 - rws_mask_79_1_output_loss: 2.8103 - rws_mask_80_0_output_loss: 2.8087 - rws_mask_80_1_output_loss: 2.8084 - rws_mask_81_0_output_loss: 2.8090 - rws_mask_81_1_output_loss: 2.8097 - rws_mask_82_0_output_loss: 2.8090 - rws_mask_82_1_output_loss: 2.8064 - rws_mask_83_0_output_loss: 2.8068 - rws_mask_83_1_output_loss: 2.8110 - rws_mask_84_0_output_loss: 2.8087 - rws_